# Lab 07 — Clean & Standardize + Join & Aggregate

**Focus Areas:** Clean & standardize (prices, dates, countries, vectorized `is_adult`) and join & aggregate (orders↔customers, per‑segment summaries)

---

## Outcomes

By the end of this lab, you will be able to:

1. Build a **reproducible cleaning pipeline** for currency, date parsing, and categorical normalization against a **reference dimension**.
2. Create **vectorized features** (e.g., `is_adult`, `high_value_user`) without `apply` loops.
3. Perform **inner joins** between orders and customers; diagnose missing keys (anti‑join), and validate cardinality to avoid fan‑out.
4. Produce **per‑segment** summaries via `groupby().agg` and export tidy Parquet artifacts for downstream LLM steps.

## Prerequisites & Setup

- Python 3.13 with `pandas`, `numpy`, `pyarrow`, `matplotlib` installed.
- JupyterLab or VS Code with Jupyter extension.

### Synthetic fallback data

Generate synthetic data for this lab.

In [ ]:
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
from pathlib import Path
import matplotlib.pyplot as plt

rng = np.random.default_rng(7)

# Customers profile
n = 1200
users = pd.DataFrame({
    'user_id': np.arange(n),
    'CustomerID': [f'C{i:05d}' for i in range(n)],
    'email': [f'user{i}@example.com' if rng.random() > 0.02 else None for i in range(n)],
    'age': rng.integers(16, 80, size=n).astype('Float64'),
    'country': rng.choice(['US','U.S.A.','usa','SG','DE','Brasil','United States', 'N/A'], size=n,
                          p=[.35,.05,.05,.15,.15,.15,.08,.02]),
    'signup_date': rng.choice(['2025-01-05','01/06/2025','06-01-2025','2025/01/07', None], size=n,
                              p=[.25,.25,.25,.2,.05]),
    'lifetime_value': rng.choice(['$1,234.50','€45,00','1,234','USD 99.95','$0.00','', None], size=n,
                                 p=[.25,.15,.25,.15,.15,.03,.02])
})

# Orders facts
m = 4000
orders = pd.DataFrame({
    'OrderID': np.arange(10_000, 10_000 + m),
    'CustomerID': rng.choice(users['CustomerID'], size=m, replace=True),
    'OrderDate': rng.choice(['2025-01-06','2025/01/07','01/08/2025','2025-01-09'], size=m),
    'ShipCountry': rng.choice(['USA','DE','SG','BR','SE'], size=m, p=[.6,.12,.12,.1,.06]),
    'Freight': rng.lognormal(mean=3.3, sigma=0.6, size=m).round(2)
})

customers = users[['CustomerID','email','age','country','signup_date','lifetime_value']].copy()

print("Users shape:", users.shape)
print("Orders shape:", orders.shape)
print("Customers shape:", customers.shape)
display(users.head())
display(orders.head())
display(customers.head())

### Country reference dimension (for normalization)

In [ ]:
country_dim = pd.DataFrame({
    'raw': ['USA','U.S.A.','United States','US','usa','U. S. A.','Brasil','BR','Germany','DE','sg','Singapore','N/A'],
    'canonical': ['USA','USA','USA','USA','USA','USA','BR','BR','DE','DE','SG','SG','UNKNOWN']
})
display(country_dim)

---

## Part A — Clean & Standardize

### A1. Inspect & guardrails

In [ ]:
users.info()

In [ ]:
users.isna().mean().sort_values(ascending=False).head(10)

**Checkpoint:** Drop rows with missing required columns (user_id, email)

In [ ]:
required = ['user_id','email']
users1 = users.dropna(subset=required)
print(f"Original users: {len(users)}, After dropna: {len(users1)}")

### A2. Normalize countries using the reference table

In [ ]:
# prep reference by normalizing its 'raw' to a matching key
norm_key = (lambda s: s.astype('string')
                     .str.replace('.','', regex=False)
                     .str.replace(' ','', regex=False)
                     .str.upper())

ref = country_dim.assign(raw_key = norm_key(country_dim['raw']))[['raw_key','canonical']]
users1 = users1.assign(country_key = norm_key(users1['country']))
users1 = users1.merge(ref, left_on='country_key', right_on='raw_key', how='left')
users1['country_norm'] = users1['canonical'].fillna('UNKNOWN')
users1.drop(columns=['raw_key','canonical'], inplace=True)
display(users1['country_norm'].value_counts().head())

### A3. Currency strings → numeric (`lifetime_value`)

In [ ]:
s = users1['lifetime_value'].astype('string').str.strip()
# Strip codes/symbols/spaces
s1 = (s.str.replace('USD','',regex=False)
        .str.replace('EUR','',regex=False)
        .str.replace('$','',regex=False)
        .str.replace('€','',regex=False)
        .str.replace('£','',regex=False)
        .str.replace(' ','',regex=False))
# Heuristics for decimal separators
mask_comma_decimal = s1.str.fullmatch(r'\d+,\d{1,2}')
mask_both = s1.str.contains(r'\d+[\.,]\d{3,}.*,\d{1,2}$')

s2 = s1.where(~mask_comma_decimal, s1.str.replace(',', '.', regex=False))
s2 = s2.where(~mask_both, s2.str.replace('.', '', regex=False).str.replace(',', '.', regex=False))
s2 = s2.str.replace(',', '', regex=False)

users1['ltv_usd'] = pd.to_numeric(s2, errors='coerce')

# Group‑wise impute by normalized country; then fallback 0.0
med = users1.groupby('country_norm')['ltv_usd'].transform('median')
users1['ltv_usd'] = users1['ltv_usd'].fillna(med).fillna(0.0)
display(users1['ltv_usd'].describe())

### A4. Dates → `datetime64[ns]`; vectorized `is_adult` and a second feature

In [ ]:
users1['signup_dt'] = pd.to_datetime(users1['signup_date'], errors='coerce', infer_datetime_format=True)
# Drop rows lacking sequencing dates if needed for later incremental logic
users2 = users1.dropna(subset=['signup_dt']).copy()

# Vectorized booleans
users2['is_adult'] = (users2['age'] >= 18)
q90 = users2['ltv_usd'].quantile(0.90)
users2['is_high_value'] = users2['ltv_usd'] >= q90

display(users2[['age','is_adult','ltv_usd','is_high_value']].head())
print(f"\nUsers after date filtering: {len(users2)}")
print(f"90th percentile LTV threshold: ${q90:.2f}")

**Checkpoint:** Why avoid `apply` here? What's the advantage of vectorization and `quantile`‑based thresholds?

**Answer:** Vectorized operations are significantly faster than `apply` because they operate on entire arrays at once using optimized C code, rather than iterating through each row with Python loops. Quantile-based thresholds automatically adapt to the data distribution, making the feature more robust to outliers and data changes over time.

### A5. Export cleaned users for joins

In [ ]:
out = Path('artifacts/clean')
out.mkdir(parents=True, exist_ok=True)
pq.write_table(pa.Table.from_pandas(users2, preserve_index=False), out / 'users2_clean.parquet')
print(f"File exists: {(out / 'users2_clean.parquet').exists()}, Rows: {len(users2)}")

---

## Part B — Join & Aggregate

### B1. Prepare orders & customers, and join

In [ ]:
# Ensure dtypes
orders['OrderDate'] = pd.to_datetime(orders['OrderDate'], errors='coerce')
customers = users2[['CustomerID','email','country_norm','signup_dt','is_adult','is_high_value']].copy()

# Inner join: realized orders with matched customers
joined = orders.merge(customers, on='CustomerID', how='inner', validate='many_to_one')
print(f"Orders: {len(orders)}, Joined: {len(joined)}")
display(joined.head())

> **Why inner?** We're building metrics about **realized orders**; missing customers are excluded here. We'll still inspect them via anti‑join next.

### B2. Diagnose missing keys (anti‑join)

In [ ]:
left = orders.merge(customers, on='CustomerID', how='left', indicator=True)
anti = left[left['_merge'] == 'left_only'][['OrderID','CustomerID']]
print(f"Unmatched orders: {len(anti)}")
display(anti.head())

**Checkpoint:** If `anti` is non‑empty, list potential causes and remediation (e.g., stale CustomerID, case mismatches, missing profile rows).

**Answer:** Potential causes include:
- Stale CustomerIDs in orders that no longer exist in the customer database
- Case sensitivity mismatches in CustomerID strings
- Missing customer profile rows due to earlier data quality filters (e.g., missing email or signup_date)
- Data sync issues between orders and customer systems
- Customer records deleted but orders retained for historical purposes

### B3. Per‑segment aggregates

In [ ]:
seg = joined.assign(
    year = joined['OrderDate'].dt.year,
    segment = np.select(
        [ joined['is_high_value'], joined['is_adult'] ],
        [ 'high_value', 'adult' ],
        default='general')
)

per_segment = (seg
    .groupby(['country_norm','segment'], as_index=False)
    .agg(orders=('OrderID','count'),
         freight_mean=('Freight','mean'),
         freight_sum=('Freight','sum'),
         customers=('CustomerID','nunique'))
    .sort_values(['country_norm','orders'], ascending=[True, False]))
display(per_segment.head())
print(f"\nTotal segment groups: {len(per_segment)}")

### B4. Per‑customer aggregates + join back to attributes

In [ ]:
per_cust = (joined
    .groupby('CustomerID', as_index=False)
    .agg(n_orders=('OrderID','count'),
         freight_mean=('Freight','mean'),
         freight_sum=('Freight','sum')))

per_cust_enriched = per_cust.merge(customers, on='CustomerID', how='left', validate='one_to_one')
display(per_cust_enriched.head())
print(f"\nUnique customers with orders: {len(per_cust_enriched)}")

### B5. Persist artifacts

In [ ]:
pq.write_table(pa.Table.from_pandas(per_segment, preserve_index=False), out / 'per_segment.parquet')
pq.write_table(pa.Table.from_pandas(per_cust_enriched, preserve_index=False), out / 'per_customer_enriched.parquet')
print("Artifacts saved successfully!")
print(f"- per_segment.parquet: {len(per_segment)} rows")
print(f"- per_customer_enriched.parquet: {len(per_cust_enriched)} rows")

---

## Part C — Wrap‑Up

### Summary and Analysis

**1. Country normalization approach:**

We used a reference dimension table (`country_dim`) to normalize country values. The approach:
- Created a normalized key by removing punctuation, spaces, and converting to uppercase
- Performed a left join between the users table and the reference dimension
- Mapped unmapped values to 'UNKNOWN'

To expand the reference table:
- Monitor unmapped values through coverage reports
- Add new raw→canonical mappings as new variations appear
- Consider maintaining this as a versioned data asset
- Implement automated alerts when unmapped rate exceeds a threshold

**2. Why inner join was the right choice:**

Inner join was appropriate because:
- We're calculating metrics for realized orders with known customer attributes
- We need complete information (both order and customer data) for accurate segmentation
- Unmatched orders were separately diagnosed via anti-join for data quality monitoring

**When left join is preferred:**
- When you want to retain all orders even without customer data (e.g., for order volume reporting)
- When analyzing data completeness issues
- When the left table is the "source of truth" and you need to preserve all its records

**3. Key metrics from per_segment:**

Let's examine two important metrics:

In [ ]:
# Top segments by total freight
top_segments = per_segment.nlargest(5, 'freight_sum')[['country_norm', 'segment', 'orders', 'freight_sum', 'customers']]
print("\nTop 5 segments by total freight:")
display(top_segments)

# Average freight per order by segment
print("\nAverage freight by segment type (across all countries):")
segment_avg = (per_segment.groupby('segment')
               .agg(total_orders=('orders', 'sum'),
                    total_freight=('freight_sum', 'sum'))
               .assign(avg_freight_per_order=lambda x: x['total_freight'] / x['total_orders']))
display(segment_avg)

**Interpretation:**

1. **Total freight by segment** reveals which customer segments generate the most shipping revenue. High-value customers likely have higher order volumes or larger shipments, making them a priority for retention and special handling.

2. **Average freight per order by segment** helps identify segment-specific shipping patterns. If high-value customers have higher per-order freight costs, this could indicate they purchase larger/heavier items or prefer premium shipping, informing pricing and logistics strategies.

---

## Stretch Goals

## 1) Reference table coverage report

In [ ]:
def norm_key_series(s: pd.Series) -> pd.Series:
    return (s.astype('string')
             .str.replace('.','', regex=False)
             .str.replace(' ','', regex=False)
             .str.upper())

ref = country_dim.assign(raw_key = norm_key_series(country_dim['raw']))[['raw_key','canonical']]
observed_keys = norm_key_series(users2['country']).value_counts()
covered = users2.merge(ref, left_on=norm_key_series(users2['country']), right_on='raw_key', how='left')
coverage_rate = 1.0 - covered['canonical'].isna().mean()
print(f"Coverage rate: {coverage_rate:.2%}")
print(f"\nObserved country keys:")
display(observed_keys)

In [ ]:
# List unmapped keys and generate suggested patch
unmapped_counts = (covered[covered['canonical'].isna()]
                   .assign(country_key=norm_key_series(covered['country']))['country_key']
                   .value_counts())
print(f"Unmapped country keys: {len(unmapped_counts)}")
display(unmapped_counts.head(10))

# Draft a patch mapping for instructors to review
suggested_patch = {k: 'UNKNOWN' for k in unmapped_counts.index}
print("\nSuggested patch dictionary:")
print(suggested_patch)

## 2) Segment stability over time (rolling trends)

In [ ]:
# Prepare daily series per segment
seg = joined.assign(
    segment = np.select([joined['is_high_value'], joined['is_adult']], ['high_value','adult'], default='general')
)
seg['day'] = seg['OrderDate'].dt.floor('D')

# Aggregate to daily totals per segment
daily = (seg.groupby(['segment','day'], as_index=False)
          .agg(freight_sum=('Freight','sum'), orders=('OrderID','count'))
          .sort_values(['segment','day']))
display(daily.head())
print(f"\nDaily segment records: {len(daily)}")

In [ ]:
# Compute rolling 7‑day sums
rolled = (daily.set_index('day')
               .groupby('segment', group_keys=False)
               .apply(lambda g: g.sort_index()
                                 .assign(freight_sum_7d=g['freight_sum'].rolling(7, min_periods=1).sum(),
                                         orders_7d=g['orders'].rolling(7, min_periods=1).sum()))
               .reset_index())
display(rolled.head(10))

In [ ]:
# Quick visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for idx, (seg_name, g) in enumerate(rolled.groupby('segment')):
    g_sorted = g.sort_values('day')
    axes[idx].plot(g_sorted['day'], g_sorted['freight_sum_7d'], marker='o')
    axes[idx].set_title(f'7-day Freight Sum — {seg_name}')
    axes[idx].set_xlabel('Date')
    axes[idx].set_ylabel('7-day Freight Sum')
    axes[idx].tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

## 3) Quantile bands (deciles/tertiles)

In [ ]:
# Create deciles on ltv_usd
metric = users2['ltv_usd'].clip(lower=0)
users2 = users2.assign(ltv_decile=pd.qcut(metric, q=10, labels=[f'd{i}' for i in range(1,11)], duplicates='drop'))
decile_counts = users2['ltv_decile'].value_counts().sort_index()
print("LTV Decile distribution:")
display(decile_counts)

# Visualize
plt.figure(figsize=(10, 5))
decile_counts.plot(kind='bar', color='steelblue')
plt.title('Customer Distribution by LTV Decile')
plt.xlabel('LTV Decile')
plt.ylabel('Number of Customers')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Use bands in downstream joins/segments
cust_attrs = users2[['CustomerID','country_norm','is_adult','ltv_decile']].copy()
joined2 = orders.merge(cust_attrs, on='CustomerID', how='inner', validate='many_to_one')
per_decile = (joined2.groupby('ltv_decile', as_index=False)
              .agg(orders=('OrderID','count'), 
                   freight_mean=('Freight','mean'), 
                   freight_sum=('Freight','sum'))
              .sort_values('ltv_decile'))
print("\nMetrics by LTV Decile:")
display(per_decile)

## 4) Join validation tests

In [ ]:
# Test 1: Anti‑join rate threshold
left = orders.merge(users2[['CustomerID']], on='CustomerID', how='left', indicator=True)
anti_rate = (left['_merge'] == 'left_only').mean()
print(f"Anti-join rate: {anti_rate:.2%}")
assert anti_rate <= 0.01, f"Anti-join rate too high: {anti_rate:.2%} (>1%)"
print("✓ Anti-join rate test passed")

In [ ]:
# Test 2: Cardinality validation (avoid fan‑out)
try:
    _ = orders.merge(users2[['CustomerID']], on='CustomerID', how='inner', validate='many_to_one')
    print("✓ Cardinality validation passed (many_to_one)")
except Exception as e:
    print(f"✗ Cardinality validation failed: {e}")

In [ ]:
# Test 3: Aggregates shouldn't be NaN in required metrics
per_segment_test = (joined.groupby('CustomerID', as_index=False)
                    .agg(n_orders=('OrderID','count'), freight_sum=('Freight','sum')))
assert per_segment_test['n_orders'].notna().all(), 'n_orders contains NaN'
assert per_segment_test['freight_sum'].notna().all(), 'freight_sum contains NaN'
print("✓ Aggregate NaN test passed")
print(f"\nAll validation tests passed successfully!")

## 5) Partitioned output

In [ ]:
# Write one file per country partition
root = out / 'per_segment'
root.mkdir(parents=True, exist_ok=True)

parts = 0
for k, g in per_segment.groupby('country_norm'):
    pq.write_table(pa.Table.from_pandas(g, preserve_index=False), root / f'country_norm={k}.parquet')
    parts += 1

partition_files = sorted(p.name for p in root.glob('country_norm=*.parquet'))[:5]
print(f"Partitions written: {parts}")
print(f"Sample partition files: {partition_files}")

In [ ]:
# Validate partition counts vs groups
n_groups = per_segment['country_norm'].nunique()
assert n_groups == parts, f"Expected {n_groups} partitions, wrote {parts}"
print(f"✓ Partition validation passed: {n_groups} groups = {parts} partitions")

## 6) LLM‑ready summary view

In [ ]:
# Build a human‑readable summary string
sample = (per_cust_enriched
          .assign(segment=np.where(per_cust_enriched['is_high_value'], 'high-value',
                                    np.where(per_cust_enriched['is_adult'], 'adult', 'general')))
          .loc[:, ['CustomerID','country_norm','n_orders','freight_sum','segment']])

summary = (sample
    .assign(summary=lambda df: (
        'Customer ' + df['CustomerID'] + ' is a ' + df['segment'] + ' buyer in ' + df['country_norm'] +
        ' with ' + df['n_orders'].astype(str) + ' orders and total freight $' + df['freight_sum'].round(2).astype(str)
    )))

print("LLM-ready customer summaries:")
display(summary.head(5))
print("\nSample summaries:")
for s in summary['summary'].head(3):
    print(f"  • {s}")

In [ ]:
# Persist the LLM view
pq.write_table(pa.Table.from_pandas(summary, preserve_index=False), out / 'per_customer_summary.parquet')
print(f"✓ LLM-ready summary saved: {len(summary)} rows")

---

## Final Summary

This notebook completed all sections of Lab 07:

**Part A - Clean & Standardize:**
- ✓ Inspected data and applied guardrails
- ✓ Normalized countries using reference dimension table
- ✓ Cleaned currency strings to numeric values
- ✓ Parsed dates and created vectorized features
- ✓ Exported cleaned artifacts

**Part B - Join & Aggregate:**
- ✓ Performed inner join between orders and customers
- ✓ Diagnosed missing keys via anti-join
- ✓ Created per-segment aggregates
- ✓ Built per-customer aggregates with attributes
- ✓ Persisted all artifacts

**Part C - Analysis:**
- ✓ Summarized country normalization approach
- ✓ Explained join strategy choices
- ✓ Analyzed key segment metrics

**Stretch Goals:**
1. ✓ Reference table coverage report with unmapped value detection
2. ✓ 7-day rolling segment stability trends with visualization
3. ✓ Quantile-based segmentation (deciles) with analysis
4. ✓ Comprehensive join validation tests
5. ✓ Partitioned parquet output by country
6. ✓ LLM-ready natural language summary generation

**Key Takeaways:**
- Vectorized operations are significantly faster than apply loops
- Reference dimension tables provide maintainable, scalable normalization
- Join validation (anti-join, cardinality checks) catches data quality issues early
- Partitioned outputs improve downstream query performance
- Quantile-based thresholds adapt to data distribution changes